In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, confusion_matrix

In [12]:
# Step 1: Read the dataset
data = pd.read_csv('Churn_Modelling.csv')  # Assuming you have downloaded the dataset

In [13]:
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [14]:
# Step 2: Distinguish the feature and target set and divide the data set into training and test sets.
X = data.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = data['Exited']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Step 3: Normalize the train and test data.
scaler = StandardScaler()
X_train_numeric = X_train.drop(['Geography', 'Gender'], axis=1)
X_test_numeric = X_test.drop(['Geography', 'Gender'], axis=1)
X_train_numeric = scaler.fit_transform(X_train_numeric)
X_test_numeric = scaler.transform(X_test_numeric)

In [17]:
# One-hot encoding for categorical variables
encoder = OneHotEncoder(sparse=False, drop='first')
X_train_categorical = encoder.fit_transform(X_train[['Geography', 'Gender']])
X_test_categorical = encoder.transform(X_test[['Geography', 'Gender']])


C:\Users\Sakshi Bodkhe\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
# Concatenate the numeric and encoded categorical features
X_train_processed = pd.concat([pd.DataFrame(X_train_numeric), pd.DataFrame(X_train_categorical)], axis=1)
X_test_processed = pd.concat([pd.DataFrame(X_test_numeric), pd.DataFrame(X_test_categorical)], axis=1)

In [19]:
# Convert labels to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [20]:
# Step 4: Initialize and build the model.
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_processed.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_processed, y_train, epochs=10, batch_size=32, validation_data=(X_test_processed, y_test))

Epoch 1/10
250/250 [==============================] - 4s 6ms/step - loss: 0.4500 - accuracy: 0.8070 - val_loss: 0.3782 - val_accuracy: 0.8470
Epoch 2/10
250/250 [==============================] - 1s 5ms/step - loss: 0.3718 - accuracy: 0.8478 - val_loss: 0.3501 - val_accuracy: 0.8535
Epoch 3/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3501 - accuracy: 0.8574 - val_loss: 0.3463 - val_accuracy: 0.8570
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3433 - accuracy: 0.8576 - val_loss: 0.3458 - val_accuracy: 0.8600
Epoch 5/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3387 - accuracy: 0.8616 - val_loss: 0.3426 - val_accuracy: 0.8620
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3356 - accuracy: 0.8615 - val_loss: 0.3412 - val_accuracy: 0.8600
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.3321 - accuracy: 0.8637 - val_loss: 0.3358 - val_accuracy: 0.8610
Epoch 

In [21]:
# Step 5: Print the accuracy score and confusion matrix
y_pred = (model.predict(X_test_processed) > 0.5).astype("int32") 
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

63/63 [==============================] - 0s 3ms/step


In [24]:
print(f"Accuracy Score: {accuracy}")
#print(accuracy)
print("Confusion Matrix:")
print(confusion)

Accuracy Score: 0.8625
Confusion Matrix:
[[1546   61]
 [ 214  179]]
